In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os

In [83]:
mainModel="../../Model/Weights/lr_dec_BiDir6200.pth"

In [ ]:
import json

# Open and read the JSON file
with open('hyperparameters.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data)

{'../../Model/Weights/lr_dec_BiDir6200.pth': {'lookback_window': 10, 'input_size': 6, 'output_size': 1, 'num_layers': 1, 'embed_dim': 64, 'hidden_size': 128, 'bidirectional': True}}


In [85]:
lookback_window = data[mainModel]['lookback_window']
input_size = data[mainModel]['input_size']
output_size = data[mainModel]['output_size']
num_layers = data[mainModel]['num_layers']
embed_dim = data[mainModel]['embed_dim']
hidden_size = data[mainModel]['hidden_size']
bidirectional = data[mainModel]['bidirectional']

In [86]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using {device}")

Using cuda


In [87]:
import torch.nn.functional as F

class AttentionMechanism(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionMechanism, self).__init__()
        self.attention_weights = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, lstm_outputs):
        # lstm_outputs: [batch_size, seq_len, hidden_dim]
        attention_scores = self.attention_weights(lstm_outputs).squeeze(-1)  # [batch_size, seq_len]
        attention_weights = F.softmax(attention_scores, dim=1)  # [batch_size, seq_len]
        weighted_output = torch.sum(lstm_outputs * attention_weights.unsqueeze(-1), dim=1)  # [batch_size, hidden_dim]
        return weighted_output, attention_weights
    
class LSTMDoubleAttentionModel(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim, num_layers=1, bidirectional=False):
        super(LSTMDoubleAttentionModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.directions = 2 if bidirectional else 1

        # Embedding layer (optional: use if input is categorical or needs projection)
        self.embedding = nn.Linear(input_dim, embed_dim)  # Replace with nn.Embedding if needed
        
        # Encoder LSTM
        self.encoder_lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, 
                                    batch_first=True, bidirectional=bidirectional)
        
        # Attention
        self.attention = AttentionMechanism(hidden_dim * self.directions)
        
        # Decoder LSTM
        self.decoder_lstm = nn.LSTM(hidden_dim * self.directions, hidden_dim, num_layers,
                                    batch_first=True, bidirectional=bidirectional)
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim * self.directions, output_dim)
    
    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        
        # Embedding
        embedded = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        
        # Encoder LSTM
        encoder_out, _ = self.encoder_lstm(embedded)  # [batch_size, seq_len, hidden_dim * directions]
        
        # Attention
        attention_out, attention_weights = self.attention(encoder_out)  # [batch_size, hidden_dim * directions]
        
        # Prepare Decoder Input (sequence length 1)
        decoder_input = attention_out.unsqueeze(1)  # [batch_size, 1, hidden_dim * directions]
        
        # Decoder LSTM
        decoder_out, _ = self.decoder_lstm(decoder_input)  # [batch_size, 1, hidden_dim * directions]
        
        # Final Prediction
        output = self.fc(decoder_out.squeeze(1))  # [batch_size, output_dim]
        
        return output, attention_weights

In [88]:
model = LSTMDoubleAttentionModel(input_size, embed_dim, hidden_size, output_size, num_layers=num_layers, bidirectional=bidirectional)

In [89]:
import sys

# Determine the parent directory of traderPackage by going two levels up
# from the current working directory (which is traderPackage/API)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

    
from traderPackage.Scaler import Scaler
from sklearn.preprocessing import StandardScaler

In [90]:

model.load_state_dict(torch.load(mainModel, weights_only=True))
model.eval()


saveName=mainModel.split("/")[-1].split(".")[0]
saveFolder=f"../../Model/Weights/scalers"

import pickle
dbfile = open(os.path.join(saveFolder,f"{saveName}/scaler_{saveName}.pkl"), 'rb')    
scaler = pickle.load(dbfile)
dbfile.close()

dbfile = open(os.path.join(saveFolder,f"{saveName}/zScalerDic_{saveName}.pkl"), 'rb')    
zScalerDic= pickle.load(dbfile)
dbfile.close()

In [91]:
dataFolder="../../Data/StandardizedData"
import pandas as pd
testData=pd.read_csv(f"{dataFolder}/CCOLA.IS.csv").drop(columns=["Pct_Change","Date"])
zScaler=zScalerDic["CCOLA.IS.csv"]

In [92]:
X=testData.iloc[1000:1000+lookback_window,:]
y=testData.iloc[1001+lookback_window,:]
X

,Open,High,Low,Close,Volume,Days_Between
1000,31.398820,32.606818,30.971935,31.898368,6125026.6,1.0
1001,31.898368,31.898368,30.890189,31.335241,6470728.0,1.0
1002,31.789376,33.605913,31.444236,33.605913,6406870.8,3.0
1003,33.633156,34.968312,33.333429,34.586839,6435908.6,1.0
1004,34.586839,35.395200,33.288016,33.796647,6130689.4,1.0
1005,33.605913,34.105460,32.697645,33.587745,5896754.6,1.0
1006,33.587745,33.769398,31.971031,33.406091,5026230.0,1.0
1007,33.106364,34.432436,32.670400,34.150873,7069326.0,3.0
1008,34.196286,34.850239,33.406091,33.406091,8375281.2,1.0
1009,33.605909,35.921995,32.970124,35.440613,10044223.2,1.0


In [93]:
def answer(model,X,zScaler,scaler):
    model.eval()
    model.to(device)
    X=X.to_numpy()
    X=zScaler.fit_transform(X)
    X=scaler.fit_transform(X)
    X=torch.tensor(np.expand_dims(X,axis=0), dtype=torch.float32)
    with torch.no_grad():
        X = X.to(device)
        outputs, att = model(X)

    zz=np.zeros((1,6))
    zz[0,3]=outputs[0,0].to("cpu").numpy()
    
    normalized=zScaler.inverse_transform(scaler.inverse_transform(zz))
    return normalized[0,3], att

In [94]:
a,att=answer(model,X,zScaler,scaler)
print("Prediction:",a," Real Value:",y.iloc[3])

Prediction: 34.76030126226647  Real Value: 38.04734090909091
